In [ ]:
##!/usr/bin/env python
"""plot_determine_HW_timing_AWS.py

Script reads in the AWS data and determines the HW timing

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Thu Jul 30 14:26:12 AEST 2020

"""

Load Packages

In [1]:
#from __future__ import division
import numpy as np
import pandas as pd
import math
import netCDF4 as nc
import sys
import os
import glob as glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
from matplotlib.collections import LineCollection
import common_functions as cf
import datetime as dt
import wrf
from scipy import stats


## AWS Data

In [2]:
# Dates to determine the heatwaves from
sdate = "2017-01-01"
edate = "2017-02-28"

# Baseline period for calculating the heatwaves
#bsdate = "1980-01-01"
#bedate = "2011-01-01" # 1 extra day for indexing purposes
bsdate = "1961-01-01"
bedate = "1991-01-01" # 1 extra day for indexing purposes

# Simulation Period
syear = 2017
smon = 1
sday = 2 
eyear = 2017
emon = 2
eday = 28  # Add an extra day so that the 27th Feb data is included
simlen = dt.datetime(eyear,emon,eday) - dt.datetime(syear,smon,sday)
nst = (simlen.days) # No. simulations days 


Data Attributes

In [3]:
awsdir = '/g/data/w97/azh561/WRF/obs/AWS_dailydata'
awsnum = ['066037']
awsnm = ['Sydney Airport']

Read in the daily data

In [4]:
tmaxfile = "%s/IDCJAC0010_066037_1800_Data_tmax.csv" %(awsdir)
tmaxdf = pd.read_csv(tmaxfile)
tminfile = "%s/IDCJAC0011_066037_1800_Data_tmin.csv" %(awsdir)
tmindf = pd.read_csv(tminfile)

In [5]:
tmaxdf.columns = ["a","No","year","month","day","tmax","tmax_no","tmax_qc"]
tmaxdf['date'] = pd.to_datetime(tmaxdf[['year','month','day']])
tmindf.columns = ["a","No","year","month","day","tmin","tmin_no","tmin_qc"]
tmindf['date'] = pd.to_datetime(tmaxdf[['year','month','day']])

Extract the baseline

In [6]:
tmax_base = tmaxdf.loc[(tmaxdf['date'] > bsdate) & (tmaxdf['date'] < bedate)]
tmin_base = tmindf.loc[(tmindf['date'] > bsdate) & (tmindf['date'] < bedate)]

In [9]:
ndays = 10956 #11322
nyrs = 30 #31
tave = np.empty((ndays),dtype=np.float64)
for dd in range(ndays):
    tave[dd] = (tmax_base['tmax'].iloc[dd] + tmin_base['tmin'].iloc[dd])/2
    
# Remove leap days
months = tmax_base['month']
days = tmax_base['day']
tave = tave[((months==2) & (days==29))==False]

Calculate the calendar day 90th percentile

In [10]:
def calc_percentile(tave,nyears,nwindow=15):
    
    windowrange=np.zeros((365,),dtype=np.bool)
    windowrange[:int(np.ceil(nwindow/2)+1)]=True
    windowrange[int(-np.floor(nwindow/2)):]=True
    windowrange=np.tile(windowrange,nyears)

    pct_calc=np.ones((365),np.float64)
    for d in range(365):
        pct_calc[d]=np.nanpercentile(tave[windowrange[:-1]==True],90,interpolation='nearest')
        windowrange=np.roll(windowrange,1)

    return pct_calc


In [11]:
pctthres = calc_percentile(tave,nyrs,15)

Get Jan-Feb 2017 data

In [12]:
tmax2017 = tmaxdf.loc[(tmaxdf['date'] > sdate) & (tmaxdf['date'] < edate)]
tmin2017 = tmindf.loc[(tmindf['date'] > sdate) & (tmindf['date'] < edate)]

In [13]:
# Calculate the daily average 
tave2017 = np.empty((nst),dtype=np.float64)
for dd in range(nst):
    tave2017[dd] = (tmax2017['tmax'].iloc[dd] + tmin2017['tmin'].iloc[dd])/2
    
# Calculate the 3-day average
tave_3days = np.empty((nst),dtype=np.float64)
for t in range(2,nst):
    tave_3days[t]=np.mean(tave2017[t-2:t+1])
    
# Calculate the EHFsig
EHFsig = np.empty((nst),dtype=np.float64)
for t in range(nst):
    EHFsig[t]=tave_3days[t] - pctthres[t]
    
# Convert EHFsig to array of zeros and ones
EHFsig[EHFsig<0]=0
EHFsig[EHFsig>0]=1

In [14]:
for t in range(nst):
    if EHFsig[t] == 1:
        print(tmax2017['date'].iloc[t])

2017-01-10 00:00:00
2017-01-11 00:00:00
2017-01-12 00:00:00
2017-01-13 00:00:00
2017-01-14 00:00:00
2017-01-15 00:00:00
2017-01-16 00:00:00
2017-01-17 00:00:00
2017-01-18 00:00:00
2017-01-19 00:00:00
2017-01-20 00:00:00
2017-01-24 00:00:00
2017-01-25 00:00:00
2017-01-30 00:00:00
2017-01-31 00:00:00
2017-02-01 00:00:00
2017-02-05 00:00:00
2017-02-06 00:00:00
2017-02-07 00:00:00
2017-02-10 00:00:00
2017-02-11 00:00:00
2017-02-12 00:00:00
2017-02-13 00:00:00
2017-02-17 00:00:00
2017-02-18 00:00:00


### For Sydney Airport the heatwaves days are:
    2017-01-11 to 2017-01-14
    2017-01-30 to 2017-02-01
    2017-02-05 to 2017-01-07
    2017-02-10 to 2017-01-12